In [1]:
import numpy as np

In [ ]:
# Network model
def MSE(y,a):
    MSE = (1/10)*( np.sum( np.power( (y-a),2)))
    return MSE

def a_y(z):
    a_y = np.divide( np.exp(z), np.sum( np.exp(z)))
    return a_y

def z_y(w_y,a_2,b_y):
    w_y = np.transpose( w_y)
    b_y = np.transpose( b_y)
    z_y = np.add( np.sum(np.multiply(w_y,a_2),axis=1), b_y)
    return z_y

def a_2_nonvect(z_2):
    if z_2<0:
        z_2=0
    return z_2
a_2 = np.vectorize( a_2_nonvect)

def z_2(w_2,a_1,b_2):
    w_2 = np.transpose( w_2)
    b_2 = np.transpose( b_2)
    z_2 = np.add( np.sum(np.multiply(w_2,a_1),axis=1), b_2)
    return z_2

def a_1_nonvect(z_1):
    if z_1<0:
        z_1=0
    return z_1
a_1 = np.vectorize( a_1_nonvect)

def z_1(w_1,x,b_1):
    w_1 = np.transpose( w_1)
    b_1 = np.transpose( b_1)
    z_1 = np.add( np.sum(np.multiply(w_1,x),axis=1), b_1)
    return z_1

# Stochastic gradiend descend
